# Inverted Pendulum example

In [1]:
from IPython.display import Image

In [39]:
from sympy import *
from sympy.physics.mechanics import *
from sympy import symbols, atan, Matrix, solve
q1, q2 = dynamicsymbols('q1:3')
q1d, q2d = dynamicsymbols('q1:3', level=1)
M,l,m,I,g = symbols('M, l, m, I, g')
from IPython.display import display_pretty, display_html, display_jpeg, display_png, display_json, display_latex, display_svg

## Model
![Pendulum Model](pendulum.png)

In [26]:
# Renaming variables as generalized coordinates
x = q1
xd = q1d
theta = q2
thetad = q2d
# Defining the elements of the lagrangian
# L = T - V
T = 0.5 * M * xd**2 + 0.5 * I * thetad**2

In [43]:
T.subs(Derivative(q1, Symbol('t')),q1d)

0.5*I*Derivative(q2(t), t)**2 + 0.5*M*Derivative(q1(t), t)**2